# NavigatorGPT - MongoDB Processing

In [113]:
import json
import pymongo
from pymongo import MongoClient, UpdateOne
from bson.objectid import Objectid

# Load the MongoDB connection string from the JSON file
PATH_TO_SECRET_JSON = '/home/jovyan/keys/mongodb_key.json'
with open(PATH_TO_SECRET_JSON) as f:
    MONGODB_URI = json.load(f)['connection_string']

# Extract the database name from the connection string
db_name = MONGODB_URI.split('/')[-1].split('?')[0]

# Create a MongoDB client using the connection string
if MONGODB_URI:
    client = MongoClient(MONGODB_URI)
    print("The MongoDB client has been initialized.")
else:
    print("Failed to initialize the MongoDB client.")

# Print the version of the pymongo package
pymongo_version = pymongo.__version__
print(f"The version of the pymongo package is {pymongo_version}")

# Connect to the specified MongoDB database and collection
db = client[db_name]
collection = db.attractions

The MongoDB client has been initialized.
The version of the pymongo package is 4.4.1


In [114]:
# Find the document with the specified Objectid in the 'attractions' collection
document = collection.find_one({"_id": Objectid("6450c55c3879cf70e53ff044")})

# Print the retrieved document
print(document)


{'_id': ObjectId('6450c55c3879cf70e53ff044'), 'title': 'Schermerhorn Building', 'key': 'SchermerhornBuilding', 'featureKey': 'landmark', 'catalog': 'nycwayfinding', 'loc': {'lat': 40.72761, 'lon': -73.99378, 'location': '376-380 Lafayette Street', 'neighborhood': 'Greenwich Village', 'postalCode': '10012', 'city': 'New York', 'state': 'NY'}, 'map': {'center': {'latitude': 40.72761, 'longitude': -73.99378}, 'zoom': 17}, 'machineTags': [{'tag': 'nycwayfinding:lp=00193'}], 'photo': {'_id': ObjectId('649340baf86d4f4d60e5f631'), 'photoId': 52920417345, 'title': 'Schermerhorn Building', 'url': 'https://live.staticflickr.com/65535/52920417345_6a30663600_t.jpg', 'width': 100, 'height': 75}, 'inventory': {'styles': ['N/A'], 'architects': ['Henry J. Hardenbergh']}, 'aliases': ['376-380 Lafayette Street Building']}


**Description:**
We have a dataset of attractions with specific fields to be updated in the database. Below is the dataset:

In [124]:
data = [

{"id": "56c0ef516e78364115ed8745", "loc.boroughCode": "BX", "loc.neighborhood": "Tremont", "loc.postalCode": "10460"},
{"id": "56c0ed266e78364115ed8743", "loc.boroughCode": "BX", "loc.neighborhood": "Tremont", "loc.postalCode": "10460"},
{"id": "56c113df6e78364115ed8760", "loc.boroughCode": "BK", "loc.neighborhood": "Prospect Heights", "loc.postalCode": "11238"},
{"id": "56c9511362a3debc2b358e84", "loc.boroughCode": "MN", "loc.neighborhood": "Chelsea", "loc.postalCode": "10001"},
{"id": "56ce835562a3debc2b358f0e", "loc.boroughCode": "BX", "loc.neighborhood": "Eastchester", "loc.postalCode": "10464"},
{"id": "56a5a5b4c60405816fdf8f46", "loc.boroughCode": "BK", "loc.neighborhood": "Downtown", "loc.postalCode": "11201"},
{"id": "56a55225c60405816fdf8ef9", "loc.boroughCode": "BK", "loc.neighborhood": "Downtown", "loc.postalCode": "11201"},
{"id": "56c93c6962a3debc2b358e80", "loc.boroughCode": "BK", "loc.neighborhood": "Downtown", "loc.postalCode": "11201"},
{"id": "56a542a3c60405816fdf8ef7", "loc.boroughCode": "BK", "loc.neighborhood": "Downtown", "loc.postalCode": "11201"},
{"id": "56a55ec9c60405816fdf8f14", "loc.boroughCode": "BK", "loc.neighborhood": "Bensonhurst", "loc.postalCode": "11214"},
{"id": "56ca833962a3debc2b358eb8", "loc.boroughCode": "QN", "loc.neighborhood": "Kew Gardens", "loc.postalCode": "11366"},
{"id": "56c6ac0e62a3debc2b358e73", "loc.boroughCode": "MN", "loc.neighborhood": "West Village", "loc.postalCode": "10014"},
{"id": "56ca88ec62a3debc2b358ec0", "loc.boroughCode": "BK", "loc.neighborhood": "Downtown", "loc.postalCode": "11201"},
{"id": "56bee52a6e78364115ed8591", "loc.boroughCode": "MN", "loc.neighborhood": "Garment District", "loc.postalCode": "10018"},
{"id": "56c3dfaf6e78364115ed87f7", "loc.boroughCode": "MN", "loc.neighborhood": "Garment District", "loc.postalCode": "10018"},
{"id": "56b5972dcd15881700bb2b1e", "loc.boroughCode": "MN", "loc.neighborhood": "Chelsea", "loc.postalCode": "10001"},
{"id": "56b6f541cd15882254355aac", "loc.boroughCode": "MN", "loc.neighborhood": "Chelsea", "loc.postalCode": "10001"},
{"id": "56b6d261cd1588295803c3a3", "loc.boroughCode": "MN", "loc.neighborhood": "Chelsea", "loc.postalCode": "10001"},
{"id": "56b68230cd15882c40f88b19", "loc.boroughCode": "MN", "loc.neighborhood": "Chelsea", "loc.postalCode": "10001"},
{"id": "56b5a05acd158828fcd0bb23", "loc.boroughCode": "MN", "loc.neighborhood": "East Harlem", "loc.postalCode": "10029"},
{"id": "56b59994cd15881f5cace870", "loc.boroughCode": "MN", "loc.neighborhood": "East Village", "loc.postalCode": "10009"},
{"id": "56b2de8dcd158824dc465b87", "loc.boroughCode": "MN", "loc.neighborhood": "East Village", "loc.postalCode": "10003"},
{"id": "56b6d261cd1588295803c3a2", "loc.boroughCode": "MN", "loc.neighborhood": "East Village", "loc.postalCode": "10009"},
{"id": "56b56a84cd1588242cef2692", "loc.boroughCode": "MN", "loc.neighborhood": "East Village", "loc.postalCode": "10009"},
{"id": "56b4334fcd15881df46140bf", "loc.boroughCode": "MN", "loc.neighborhood": "East Village", "loc.postalCode": "10009"},
{"id": "56b7b9eccd15882d3ca4a1af", "loc.boroughCode": "MN", "loc.neighborhood": "East Village", "loc.postalCode": "10009"},
{"id": "56ac53bccd15883f548f6f8f", "loc.boroughCode": "BX", "loc.neighborhood": "Eastchester", "loc.postalCode": "10464"},
{"id": "56b570e2cd15880120dc35c7", "loc.boroughCode": "MN", "loc.neighborhood": "Garment District", "loc.postalCode": "10018"},
{"id": "56b2c02bcd158817288a84e1", "loc.boroughCode": "MN", "loc.neighborhood": "Garment District", "loc.postalCode": "10018"},
{"id": "56b570e2cd15880120dc35c5", "loc.boroughCode": "MN", "loc.neighborhood": "Washington Heights", "loc.postalCode": "10040"},
{"id": "56b570e2cd15880120dc35c6", "loc.boroughCode": "MN", "loc.neighborhood": "Washington Heights", "loc.postalCode": "10040"},
{"id": "56b6d261cd1588295803c3a4", "loc.boroughCode": "MN", "loc.neighborhood": "Washington Heights", "loc.postalCode": "10040"},
{"id": "56b41ae4cd15882dc05a16c5", "loc.boroughCode": "MN", "loc.neighborhood": "West Village", "loc.postalCode": "10014"},
{"id": "56bfc69bcd158821548019cc", "loc.boroughCode": "QN", "loc.neighborhood": "Woodside", "loc.postalCode": "11377"},
{"id": "56bffc9ecd15882c0c671241", "loc.boroughCode": "QN", "loc.neighborhood": "Woodside", "loc.postalCode": "11377"}





]

**Description:**
Now, we'll loop through the dataset and update the records in the MongoDB collection.

In [125]:
# Initialize counters and operations list
totalRecords = len(data)
recordsUpdated = 0
operations = []

# Prepare bulk update operations for each record in data
for item in data:
    update_fields = {}

    # Iterate over all keys in the item
    for key, value in item.items():
        # Check if the value is a dictionary (nested dictionary in MongoDB)
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                update_fields[f"{key}.{sub_key}"] = sub_value
        else:
            update_fields[key] = value

    operations.append(
        UpdateOne(
            {"_id": ObjectId(item['id'])},  # corrected here
            {"$set": update_fields},
            upsert=True
        )
    )

# Execute the update operations in bulk
result = collection.bulk_write(operations)

# Calculate the total records updated
recordsUpdated = result.modified_count + len(result.upserted_ids)

# Output the results
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")


Total records: 35
Records updated: 35


**Description:**
Print out the results, showing the total number of records processed and the number of records updated.

In [126]:
print(f"Total records: {totalRecords}")
print(f"Records updated: {recordsUpdated}")

Total records: 35
Records updated: 35
